# Retrieval Augmented Generation Pyhton Demo #

The purpose of this demo is to show my lessons learned about RAG and to show my skills in pyhton.

This Demo uses simple local components to build a fully functional Retrieval-Augmented Generation (RAG) system without requiring cloud APIs.

## Code: ##

In [37]:
# This model turns text chunks into vector embeddings for similarity search

from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [38]:
# Breaking the document into overlapping chunks that fit inside the LLM's token limit

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Loading the Wikipedia page text and splitting it
loader = TextLoader("example.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Local embeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS vectorstore allows fast similarity-based retrieval later
vectorstore = FAISS.from_documents(docs, embedding)



In [41]:
# Using flan-t5-base for efficient generation which needs no API key unlike OpenAI which I had problems with a little

from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

print("Downloading flan-t5-base...")
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)
print("Model ready.")

# Wraping HuggingFace pipeline in LangChain's interface
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Connecting the retriever + LLM to perform context-aware answers
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


Model ready.


In [42]:
# Creating a simple ask function for questions
def ask(query: str):
    result = qa_chain({"query": query})
    print("Answer:", result["result"])
    
# Uncomment the following lines to get sources
    #for i, doc in enumerate(result["source_documents"], 1):
        #print(f"\n--- Source {i} ---\n{doc.page_content[:500]}...")


## Now some simple questions to test the RAG ##

In [33]:
ask("Where is Kongsberg from?")

Answer: Norway


In [44]:
ask("Who is the CEO of Kongsberg")

Answer: Geir Hy


In [45]:
ask("What 4 braches are there to Kongsberg?")

Answer: Kongsberg Maritime Kongsberg Defence & Aerospace Kongsberg Discovery Kongsberg Digital
